In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

In [2]:
path = "assignment_2_data"
anime = pd.read_csv(f'{path}/assignment_2_anime.csv')
rtrain = pd.read_csv(f"{path}/assignment_2_ratings_train.csv")
rtest = pd.read_csv(f"{path}/assignment_2_ratings_test.csv")

# preprocess rtrain as discussed in explore.ipynb
rtrain = rtrain.drop_duplicates(subset=['user_id', 'anime_id'], keep='last')
rtest = rtest.drop_duplicates(subset=['user_id', 'anime_id'], keep='last')



# build similiarity matrix

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import itertools

In [4]:
onehot = pd.get_dummies(anime['type'])
from scipy.sparse import csr_matrix
sparse_one = csr_matrix(onehot)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
vectorizer1 = TfidfVectorizer()
vectorizer2_name = TfidfVectorizer()


name_matrix = vectorizer2_name.fit_transform(anime['name'].values)
# use svd to get the dimension reduction
k = 100
svd = TruncatedSVD(n_components=k, random_state=42)
reduced_matrix= svd.fit_transform(name_matrix)
# use another vectorizer to calculate genre
anime['genre'] = anime['genre'].fillna('')
genre_matrix = vectorizer1.fit_transform(anime['genre']) # 47 genre 


In [6]:
from scipy.sparse import hstack
feature_matrix = hstack((reduced_matrix,genre_matrix, onehot.values))

In [58]:
weights_1 = np.full((k,), 0.7)
weights_2 = np.full((47,), 0.2)
weights_3 = np.full((6,), 0.1)
weights = np.concatenate((weights_1, weights_2,weights_3))
weighted_matrix = feature_matrix.multiply(weights)

# the weighted of name genre type are 0.1 0.8 0.1

In [59]:
similarity_matrix = cosine_similarity(weighted_matrix)

In [37]:
# similarity_matrix = cosine_similarity(feature_matrix)

# make prediction

In [9]:
def get_index_from_anime_id(anime_id, anime_df):
    index = anime_df[anime_df['anime_id'] == anime_id].index[0]
    return index

In [10]:
def get_index(animes_col,anime):
    ls = []
    for anime_id in animes_col:
        ls.append(get_index_from_anime_id(anime_id,anime))
    return ls
        

In [12]:
def content_based_prediction(user_id, anime_id, user_ratings, anime_df, similarities):
    "given the anime_id, user_id in the test dataset"
    # Get the index of the anime in the anime_df
    anime_index = anime_df[anime_df['anime_id'] == anime_id].index[0]

    # Get the ratings of the user, pandaframe
    user_rated_animes = user_ratings[user_ratings['user_id'] == user_id]

    # Calculate the weighted sum of the user's ratings and the genre similarity
    #change the index
    weighted_sum = np.sum(user_rated_animes['rating'].values * similarities[anime_index][user_rated_animes['index'].values])

    # Normalize the weighted sum by the sum of genre similarities
    prediction = weighted_sum / np.sum(similarities[anime_index][user_rated_animes['index'].values])

    return prediction

In [13]:
anime = anime.reset_index()

In [14]:
index_train = pd.merge(rtrain, anime[['index','anime_id']], on='anime_id', how='left')

In [15]:
rtest.head(1)

,user_id,anime_id,rating
0,44017,13161,4


In [16]:
content_based_prediction(44017, 13161, index_train, anime, similarity_matrix)

7.533738349380632

In [1]:
# Create a new DataFrame for storing the results
test_results = pd.DataFrame(columns=[ 'Ground_Truth', 'content_pred'])


# Calculate the combined predictions for each (user_id, anime_id) pair in the test dataset
for user_id, anime_id, true_rating in rtest.values: 
    # use rtest.head(5000 as example)
    
    #we use 0 first, 
    ### ??? if 
    #collab_pred = algo.predict(user_id, anime_id).est
    content_pred = content_based_prediction(user_id, anime_id, index_train, anime, similarity_matrix)
    
  
    test_results = test_results.append({
        'Ground_Truth':true_rating,
        'content_pred': content_pred,
        }, 
        ignore_index = True
    )

print(test_results.head())

NameError: name 'pd' is not defined

In [63]:
from sklearn.metrics import mean_squared_error
import numpy as np
# Replace any NaN values in the 'Ground_Truth' and 'content_pred' columns with zeros
ground_truth = np.nan_to_num(test_results['Ground_Truth'], nan=0.0)
content_pred = np.nan_to_num(test_results['content_pred'], nan=0.0)

# Calculate the MSE between the modified 'Ground_Truth' and 'content_pred' columns
mse = mean_squared_error(ground_truth, content_pred)
print("MSE:", mse)

MSE: 1.7582105691605658


different weightage
0.9 0.05  0.05  MSE 2.1
0.7 0.2 0.1 MSE: 1.76
0.4 0.4 0.1 MSE: 1.76
            


